# Day 1: Calorie Counting

The Elves take turns writing down the number of Calories contained by the various meals, snacks, rations, etc. that they've brought with them, one item per line. Each Elf separates their own inventory from the previous Elf's inventory (if any) by a blank line.

In [76]:
let partitionBy isSeparator lines = 
    let rec partition groups group xs =
        match xs with
        | [] -> group::groups
        | head :: tail when isSeparator head -> partition (group::groups) [] tail
        | head :: tail -> partition groups (head::group) tail
    partition [] [] lines
    
let totalCalories inventory =
    inventory
    |> Seq.map Int32.Parse
    |> Seq.sum

let caloriesByItems = 
    File.ReadAllLines(@"Day1.txt") 
    |> Array.toList

let caloriesByElf = 
    caloriesByItems
    |> partitionBy String.IsNullOrEmpty
    |> Seq.map totalCalories

Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?

In [77]:
Seq.max caloriesByElf

69310

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [78]:
caloriesByElf
    |> Seq.sortDescending
    |> Seq.take 3
    |> Seq.sum

206104

# Day 2: Rock Paper Scissors

Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).

In [79]:
type RPS = Rock | Paper | Scissors

let rpsRules = Map[ Rock, Scissors; Paper, Rock; Scissors, Paper; ]

let scoreShape = Map[ Rock, 1; Paper, 2; Scissors, 3; ]

let scoreRound opponent myShape =
    match opponent with
    | x when x = myShape -> 3
    | x when rpsRules[x] = myShape -> 0
    | _ -> 6

let rounds = 
    File.ReadAllLines("Day2.txt")
    |> Seq.map (fun round -> round.Split(" "))
    |> Seq.map (fun [|a; b|] -> (a, b))
    
let scoreByShapes (p1, p2) =
    scoreShape[p2] + scoreRound p1 p2

Find the total score by shape.

In [80]:
let p1ToShape = Map[ "A", Rock; "B", Paper; "C", Scissors; ]

let p2ToShape = Map[ "X", Rock; "Y", Paper; "Z", Scissors; ]

let getShapesByString (p1, p2) =
    (p1ToShape[p1], p2ToShape[p2])
    
rounds 
    |> Seq.map getShapesByString 
    |> Seq.map scoreByShapes
    |> Seq.sum

17189

Find the total score by round result.

In [81]:
type RoundResult =  Win | Loss | Draw

let p2ToRoundResult = Map["X", Loss; "Y", Draw; "Z", Win; ]

let swap (a, b) = (b, a)

let invert = Map.toSeq >> Seq.map swap >> Map.ofSeq
    
let getShapeByResult opponent desiredResult =
    match desiredResult with
    | Win -> invert(rpsRules)[opponent]
    | Draw -> opponent
    | Loss -> rpsRules[opponent]
    
let getShapesByResult (p1, p2) =
    let p1Shape = p1ToShape[p1]
    let p2Shape = p2ToRoundResult[p2] |> getShapeByResult p1Shape
    (p1Shape, p2Shape)
    
rounds 
    |> Seq.map getShapesByResult 
    |> Seq.map scoreByShapes
    |> Seq.sum

13490

# Day 3: Rucksack Reorganization

One Elf has the important job of loading all of the rucksacks with supplies for the jungle journey. To help prioritize item rearrangement, every item type can be converted to a priority:

- Lowercase item types a through z have priorities 1 through 26.
- Uppercase item types A through Z have priorities 27 through 52.

In [82]:
let priorities =
    [ 'a' .. 'z' ] @ [ 'A' .. 'Z' ]

let getPriority priorities item =
    priorities
    |> Seq.findIndex (fun x -> x = item) 
    |> (+) 1

let findCommonItems common items =
    Set.intersect (set common) (set items)
    |> Set.toArray
    
let sumPriorities inventories =
    inventories
    |> Seq.map (Seq.reduce findCommonItems)
    |> Seq.map Seq.head
    |> Seq.map (getPriority priorities)
    |> Seq.sum

let rucksacks = 
    File.ReadAllLines("Day3.txt") 
    |> Seq.map (fun inventory -> inventory.ToCharArray())

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [83]:
let priorityByCommonItem =
    Seq.map (Seq.splitInto 2)
    >> sumPriorities
    
priorityByCommonItem rucksacks

8018

Find the item type that corresponds to the badges of each three-Elf group. What is the sum of the priorities of those item types?

In [84]:
let priorityByBadge =
    Seq.chunkBySize 3
    >> sumPriorities

priorityByBadge rucksacks

2518

# Day 4: Camp Cleanup

Space needs to be cleared before the last supplies can be unloaded from the ships, and so several Elves have been assigned the job of cleaning up sections of the camp. Every section has a unique ID number, and each Elf is assigned a range of section IDs.

To try to quickly find overlaps and reduce duplicated effort, the Elves pair up and make a big list of the section assignments for each pair (your puzzle input).

In [95]:
let toRange (min, max) =
    [min..max]

let firstPair xs =
    xs 
    |> Seq.pairwise
    |> Seq.head

let createSection (section: string) =
    section.Split '-'
    |> Seq.map Int32.Parse
    |> firstPair
    |> toRange

let createSectionPair (sectionPair: string) =
    sectionPair.Split ','
    |> Seq.map createSection
    |> firstPair

let sectionPairs =
    File.ReadAllLines("Day4.txt")
    |> Seq.map createSectionPair

In how many assignment pairs does one range fully contain the other?

In [96]:
let fullyOverlaps (elf1, elf2) =
    let section1 = set elf1
    let section2 = set elf2
    Set.intersect section1 section2 = section1
    || Set.intersect section1 section2 = section2

sectionPairs
    |> Seq.filter fullyOverlaps
    |> Seq.length

464

In how many assignment pairs do the ranges overlap?

In [97]:
let overlaps (elf1, elf2) =
    Set.intersect (set elf1) (set elf2)
    |> Seq.length

sectionPairs
    |> Seq.map overlaps
    |> Seq.filter (fun len -> len > 0)
    |> Seq.length

770